# Chapter 2 - Exercise 2: ECM Composition and Mechanical Properties

**VUB Master Bioengineering - Biofabrication Course**

---

## Learning Objectives
- Analyze the relationship between ECM composition and tissue mechanical properties
- Explore tissue-specific ECM distributions (bone, cartilage, skin, etc.)
- Predict Young's modulus from molecular composition
- Design ECM formulations for specific tissue engineering applications
- Practice data analysis and correlation techniques in Python

## Background Theory

From **Chapter 2.2**, we learned that the **extracellular matrix (ECM)** is composed of:

- **Glycosaminoglycans (GAGs)** - provide hydration and compressive strength
- **Proteins** (collagen, elastin, fibronectin) - provide tensile strength and elasticity  
- **Proteoglycans** - regulate hydration and cell signaling

The **composition** varies dramatically between tissues, giving each organ its unique mechanical properties. **Table 2.5** shows Young's modulus values ranging from brain (240 × 10⁻⁶ MPa) to bone (18 × 10³ MPa).

This exercise explores quantitative relationships between ECM composition and mechanical properties.

---

## Instructions for Students

1. **Save a copy**: File → Save a copy in Drive
2. **Run cells in order**: Use Shift+Enter or click the play button
3. **Modify parameters**: Look for comments with `# MODIFY THIS`
4. **Answer questions**: Complete the analysis section at the end
5. **Experiment**: Try different compositions and observe changes

# Section 1: Setup and Imports

Run this cell first to install required packages and import libraries.

In [ ]:
# Install required packages (run this cell first!)
!pip install matplotlib seaborn pandas numpy scipy plotly --quiet

# Import libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from scipy import stats
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

# Set plotting style
plt.style.use('default')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (14, 10)
plt.rcParams['font.size'] = 11

print("Setup complete! Ready to analyze ECM composition and mechanical properties.")
print("Packages loaded: numpy, matplotlib, pandas, seaborn, scipy, plotly")

# Section 2: ECM Composition Database

Based on **Tables 2.3, 2.4, and 2.5** from Chapter 2, we'll create a comprehensive database of tissue compositions and properties.

In [ ]:
# ECM Composition Database (based on Chapter 2 tables)
# Values represent relative concentrations (normalized to total protein content)

ecm_database = {
    'Bone': {
        'collagen_I': 0.90,
        'collagen_II': 0.0,
        'collagen_IV': 0.0,
        'elastin': 0.02,
        'fibronectin': 0.03,
        'laminin': 0.0,
        'hyaluronic_acid': 0.01,
        'chondroitin_sulfate': 0.02,
        'dermatan_sulfate': 0.01,
        'heparan_sulfate': 0.01,
        'hydroxyapatite': 0.65,  # Mineral content
        'water_content': 0.20,
        'youngs_modulus_MPa': 18000  # From Table 2.5
    },
    'Cartilage': {
        'collagen_I': 0.05,
        'collagen_II': 0.60,
        'collagen_IV': 0.0,
        'elastin': 0.01,
        'fibronectin': 0.02,
        'laminin': 0.0,
        'hyaluronic_acid': 0.05,
        'chondroitin_sulfate': 0.20,
        'dermatan_sulfate': 0.02,
        'heparan_sulfate': 0.02,
        'hydroxyapatite': 0.0,
        'water_content': 0.80,
        'youngs_modulus_MPa': 1.2
    },
    'Skin': {
        'collagen_I': 0.75,
        'collagen_II': 0.0,
        'collagen_IV': 0.02,
        'elastin': 0.15,
        'fibronectin': 0.05,
        'laminin': 0.01,
        'hyaluronic_acid': 0.01,
        'chondroitin_sulfate': 0.01,
        'dermatan_sulfate': 0.08,
        'heparan_sulfate': 0.01,
        'hydroxyapatite': 0.0,
        'water_content': 0.70,
        'youngs_modulus_MPa': 0.421
    },
    'Muscle': {
        'collagen_I': 0.40,
        'collagen_II': 0.0,
        'collagen_IV': 0.05,
        'elastin': 0.03,
        'fibronectin': 0.10,
        'laminin': 0.15,
        'hyaluronic_acid': 0.02,
        'chondroitin_sulfate': 0.05,
        'dermatan_sulfate': 0.05,
        'heparan_sulfate': 0.10,
        'hydroxyapatite': 0.0,
        'water_content': 0.75,
        'youngs_modulus_MPa': 0.018
    },
    'Brain': {
        'collagen_I': 0.10,
        'collagen_II': 0.0,
        'collagen_IV': 0.20,
        'elastin': 0.01,
        'fibronectin': 0.05,
        'laminin': 0.30,
        'hyaluronic_acid': 0.15,
        'chondroitin_sulfate': 0.08,
        'dermatan_sulfate': 0.02,
        'heparan_sulfate': 0.08,
        'hydroxyapatite': 0.0,
        'water_content': 0.85,
        'youngs_modulus_MPa': 0.00024
    },
    'Blood_Vessel': {
        'collagen_I': 0.50,
        'collagen_II': 0.0,
        'collagen_IV': 0.10,
        'elastin': 0.25,
        'fibronectin': 0.08,
        'laminin': 0.05,
        'hyaluronic_acid': 0.01,
        'chondroitin_sulfate': 0.01,
        'dermatan_sulfate': 0.03,
        'heparan_sulfate': 0.02,
        'hydroxyapatite': 0.0,
        'water_content': 0.70,
        'youngs_modulus_MPa': 0.5  # Estimated
    }
}

# Convert to DataFrame for analysis
ecm_df = pd.DataFrame(ecm_database).T

print("ECM Composition Database Created")
print(f"Tissues analyzed: {list(ecm_database.keys())}")
print(f"ECM components tracked: {len(ecm_df.columns)} parameters")
print("\nDatabase Preview:")
print(ecm_df.head())

# Section 3: ECM Composition Analysis

Let's visualize how ECM composition varies across different tissues.

In [ ]:
# Create comprehensive ECM composition visualization
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('ECM Composition Analysis Across Tissues', fontsize=16, fontweight='bold')

# Plot 1: Collagen types distribution
collagen_cols = ['collagen_I', 'collagen_II', 'collagen_IV']
collagen_data = ecm_df[collagen_cols]

collagen_data.plot(kind='bar', ax=axes[0,0], width=0.8)
axes[0,0].set_title('Collagen Types Distribution', fontweight='bold')
axes[0,0].set_ylabel('Relative Concentration', fontweight='bold')
axes[0,0].tick_params(axis='x', rotation=45)
axes[0,0].legend(title='Collagen Types')
axes[0,0].grid(True, alpha=0.3)

# Plot 2: GAGs distribution
gag_cols = ['hyaluronic_acid', 'chondroitin_sulfate', 'dermatan_sulfate', 'heparan_sulfate']
gag_data = ecm_df[gag_cols]

gag_data.plot(kind='bar', ax=axes[0,1], width=0.8, colormap='viridis')
axes[0,1].set_title('Glycosaminoglycans (GAGs) Distribution', fontweight='bold')
axes[0,1].set_ylabel('Relative Concentration', fontweight='bold')
axes[0,1].tick_params(axis='x', rotation=45)
axes[0,1].legend(title='GAG Types', bbox_to_anchor=(1.05, 1), loc='upper left')
axes[0,1].grid(True, alpha=0.3)

# Plot 3: Structural proteins
protein_cols = ['elastin', 'fibronectin', 'laminin']
protein_data = ecm_df[protein_cols]

protein_data.plot(kind='bar', ax=axes[1,0], width=0.8, colormap='plasma')
axes[1,0].set_title('Structural Proteins Distribution', fontweight='bold')
axes[1,0].set_ylabel('Relative Concentration', fontweight='bold')
axes[1,0].tick_params(axis='x', rotation=45)
axes[1,0].legend(title='Protein Types')
axes[1,0].grid(True, alpha=0.3)

# Plot 4: Water content vs Young's modulus
scatter = axes[1,1].scatter(ecm_df['water_content'], 
                           np.log10(ecm_df['youngs_modulus_MPa']),
                           s=100, alpha=0.7, c=range(len(ecm_df)), cmap='tab10')

# Add tissue labels
for i, tissue in enumerate(ecm_df.index):
    axes[1,1].annotate(tissue, 
                       (ecm_df.loc[tissue, 'water_content'], 
                        np.log10(ecm_df.loc[tissue, 'youngs_modulus_MPa'])),
                       xytext=(5, 5), textcoords='offset points', fontsize=9)

axes[1,1].set_xlabel('Water Content', fontweight='bold')
axes[1,1].set_ylabel('Log₁₀(Young\'s Modulus [MPa])', fontweight='bold')
axes[1,1].set_title('Water Content vs Mechanical Stiffness', fontweight='bold')
axes[1,1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("ECM composition analysis complete!")
print("Key observations:")
print("• Bone has highest collagen I and mineral content")
print("• Cartilage is rich in collagen II and chondroitin sulfate")
print("• Brain has high water content and low stiffness")
print("• Blood vessels have significant elastin for flexibility")

# Section 4: Correlation Analysis

Let's analyze which ECM components most strongly correlate with mechanical properties.

In [ ]:
# Calculate correlations with Young's modulus
correlations = ecm_df.corr()['youngs_modulus_MPa'].drop('youngs_modulus_MPa')
correlations = correlations.sort_values(key=abs, ascending=False)

# Create correlation heatmap
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Plot 1: Correlation bar chart
colors = ['red' if x < 0 else 'blue' for x in correlations.values]
bars = axes[0].bar(range(len(correlations)), correlations.values, color=colors, alpha=0.7)
axes[0].set_xticks(range(len(correlations)))
axes[0].set_xticklabels(correlations.index, rotation=45, ha='right')
axes[0].set_ylabel('Correlation with Young\'s Modulus', fontweight='bold')
axes[0].set_title('ECM Component Correlations with Stiffness', fontweight='bold')
axes[0].axhline(y=0, color='black', linestyle='-', alpha=0.3)
axes[0].grid(True, alpha=0.3)

# Add value labels on bars
for bar, value in zip(bars, correlations.values):
    height = bar.get_height()
    axes[0].text(bar.get_x() + bar.get_width()/2, 
                height + (0.02 if height >= 0 else -0.05),
                f'{value:.2f}', ha='center', va='bottom' if height >= 0 else 'top',
                fontweight='bold', fontsize=9)

# Plot 2: Full correlation matrix heatmap
# Select key components for readability
key_components = ['collagen_I', 'collagen_II', 'elastin', 'hyaluronic_acid', 
                  'chondroitin_sulfate', 'water_content', 'hydroxyapatite', 'youngs_modulus_MPa']
corr_matrix = ecm_df[key_components].corr()

im = axes[1].imshow(corr_matrix, cmap='RdBu_r', aspect='auto', vmin=-1, vmax=1)
axes[1].set_xticks(range(len(key_components)))
axes[1].set_yticks(range(len(key_components)))
axes[1].set_xticklabels(key_components, rotation=45, ha='right')
axes[1].set_yticklabels(key_components)
axes[1].set_title('ECM Component Correlation Matrix', fontweight='bold')

# Add correlation values to heatmap
for i in range(len(key_components)):
    for j in range(len(key_components)):
        text = axes[1].text(j, i, f'{corr_matrix.iloc[i, j]:.2f}',
                           ha="center", va="center", color="white" if abs(corr_matrix.iloc[i, j]) > 0.5 else "black",
                           fontweight='bold', fontsize=8)

# Add colorbar
cbar = plt.colorbar(im, ax=axes[1], shrink=0.8)
cbar.set_label('Correlation Coefficient', fontweight='bold')

plt.tight_layout()
plt.show()

print("Correlation Analysis Results:")
print("\nStrongest positive correlations with stiffness:")
positive_corr = correlations[correlations > 0].head(3)
for component, corr in positive_corr.items():
    print(f"  • {component}: {corr:.3f}")

print("\nStrongest negative correlations with stiffness:")
negative_corr = correlations[correlations < 0].head(3)
for component, corr in negative_corr.items():
    print(f"  • {component}: {corr:.3f}")

# Section 5: Young's Modulus Predictor

Create a simple model to predict tissue stiffness from ECM composition.

In [ ]:
# Simple linear model for Young's modulus prediction
def predict_youngs_modulus(composition):
    """
    Predict Young's modulus from ECM composition using empirical relationships.
    
    Based on observed correlations:
    - Collagen I and hydroxyapatite increase stiffness
    - Water content decreases stiffness
    - Elastin provides moderate stiffness with flexibility
    """
    
    # Empirical coefficients (simplified model)
    base_stiffness = 0.001  # Base value for soft tissues
    
    # Positive contributors to stiffness
    collagen_I_effect = composition.get('collagen_I', 0) * 2.0
    collagen_II_effect = composition.get('collagen_II', 0) * 0.5
    hydroxyapatite_effect = composition.get('hydroxyapatite', 0) * 30000  # Major effect
    elastin_effect = composition.get('elastin', 0) * 1.0
    
    # Negative contributors (softening effects)
    water_effect = -(composition.get('water_content', 0) - 0.2) * 0.5  # Penalty above 20%
    gag_effect = -(composition.get('hyaluronic_acid', 0) + 
                   composition.get('chondroitin_sulfate', 0)) * 0.1
    
    predicted_modulus = (base_stiffness + 
                        collagen_I_effect + 
                        collagen_II_effect + 
                        hydroxyapatite_effect + 
                        elastin_effect + 
                        water_effect + 
                        gag_effect)
    
    return max(predicted_modulus, 0.0001)  # Ensure positive value

# Test predictions on known tissues
predictions = []
actual_values = []
tissue_names = []

for tissue, composition in ecm_database.items():
    predicted = predict_youngs_modulus(composition)
    actual = composition['youngs_modulus_MPa']
    
    predictions.append(predicted)
    actual_values.append(actual)
    tissue_names.append(tissue)
    
    print(f"{tissue:12} - Predicted: {predicted:8.3f} MPa, Actual: {actual:8.3f} MPa")

# Calculate prediction accuracy
predictions = np.array(predictions)
actual_values = np.array(actual_values)

# Use log scale for better comparison across orders of magnitude
log_predictions = np.log10(predictions)
log_actual = np.log10(actual_values)

correlation, p_value = stats.pearsonr(log_predictions, log_actual)
print(f"\nModel Performance (log scale):")
print(f"Correlation coefficient: {correlation:.3f}")
print(f"P-value: {p_value:.3f}")

# Visualization
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Plot 1: Predicted vs Actual (log scale)
axes[0].scatter(log_actual, log_predictions, s=100, alpha=0.7)
for i, tissue in enumerate(tissue_names):
    axes[0].annotate(tissue, (log_actual[i], log_predictions[i]), 
                    xytext=(5, 5), textcoords='offset points', fontsize=9)

# Perfect prediction line
min_val, max_val = min(log_actual.min(), log_predictions.min()), max(log_actual.max(), log_predictions.max())
axes[0].plot([min_val, max_val], [min_val, max_val], 'r--', alpha=0.8, label='Perfect Prediction')

axes[0].set_xlabel('Log₁₀(Actual Young\'s Modulus [MPa])', fontweight='bold')
axes[0].set_ylabel('Log₁₀(Predicted Young\'s Modulus [MPa])', fontweight='bold')
axes[0].set_title(f'Model Prediction Accuracy\n(r = {correlation:.3f})', fontweight='bold')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Plot 2: Prediction errors
errors = log_predictions - log_actual
bars = axes[1].bar(tissue_names, errors, alpha=0.7, 
                   color=['red' if e > 0 else 'blue' for e in errors])
axes[1].set_ylabel('Log₁₀(Prediction Error)', fontweight='bold')
axes[1].set_title('Prediction Errors by Tissue', fontweight='bold')
axes[1].tick_params(axis='x', rotation=45)
axes[1].axhline(y=0, color='black', linestyle='-', alpha=0.3)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Section 6: ECM Designer Tool (Students: Modify This!)

Design your own ECM composition and predict its mechanical properties.

In [ ]:
# =============================================================================
# STUDENT ECM DESIGNER - MODIFY THESE VALUES!
# =============================================================================

# Design your custom ECM composition
custom_ecm = {
    'collagen_I': 0.60,          # MODIFY THIS (0-1)
    'collagen_II': 0.10,         # MODIFY THIS (0-1)
    'collagen_IV': 0.05,         # MODIFY THIS (0-1)
    'elastin': 0.10,             # MODIFY THIS (0-1)
    'fibronectin': 0.05,         # MODIFY THIS (0-1)
    'laminin': 0.02,             # MODIFY THIS (0-1)
    'hyaluronic_acid': 0.03,     # MODIFY THIS (0-1)
    'chondroitin_sulfate': 0.03, # MODIFY THIS (0-1)
    'dermatan_sulfate': 0.01,    # MODIFY THIS (0-1)
    'heparan_sulfate': 0.01,     # MODIFY THIS (0-1)
    'hydroxyapatite': 0.0,       # MODIFY THIS (0-1) - for mineralized tissues
    'water_content': 0.75        # MODIFY THIS (0-1)
}

# Target application (modify this too!)
target_application = "Custom Tissue"  # MODIFY THIS

# =============================================================================

# Analyze your custom ECM
predicted_modulus = predict_youngs_modulus(custom_ecm)

print(f"ECM Design Analysis for: {target_application}")
print("=" * 50)
print(f"Predicted Young's Modulus: {predicted_modulus:.4f} MPa")
print(f"Predicted stiffness category: ", end="")

if predicted_modulus > 1000:
    print("Very stiff (bone-like)")
elif predicted_modulus > 1:
    print("Moderately stiff (cartilage-like)")
elif predicted_modulus > 0.1:
    print("Soft (skin-like)")
elif predicted_modulus > 0.01:
    print("Very soft (muscle-like)")
else:
    print("Ultra-soft (brain-like)")

# Compare with natural tissues
print("\nComparison with natural tissues:")
for tissue, data in ecm_database.items():
    actual_modulus = data['youngs_modulus_MPa']
    ratio = predicted_modulus / actual_modulus
    if 0.5 <= ratio <= 2.0:  # Similar stiffness
        print(f"  • Similar to {tissue} ({actual_modulus:.4f} MPa)")

# Composition breakdown
print("\nComposition Analysis:")
total_protein = sum([custom_ecm[k] for k in custom_ecm.keys() if k not in ['water_content', 'hydroxyapatite']])
print(f"  • Total protein content: {total_protein:.2f}")
print(f"  • Water content: {custom_ecm['water_content']:.2f}")
print(f"  • Mineral content: {custom_ecm['hydroxyapatite']:.2f}")

# Dominant components
sorted_components = sorted(custom_ecm.items(), key=lambda x: x[1], reverse=True)
print(f"\nTop 3 components:")
for i, (component, value) in enumerate(sorted_components[:3]):
    if value > 0.01:  # Only show significant components
        print(f"  {i+1}. {component}: {value:.2f}")

# Visualize your design
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Plot 1: Composition pie chart
significant_components = {k: v for k, v in custom_ecm.items() if v > 0.01}
if len(significant_components) > 8:  # Combine small components
    sorted_comp = sorted(significant_components.items(), key=lambda x: x[1], reverse=True)
    major_comp = dict(sorted_comp[:7])
    other_value = sum([v for k, v in sorted_comp[7:]])
    if other_value > 0:
        major_comp['Others'] = other_value
    significant_components = major_comp

wedges, texts, autotexts = axes[0].pie(significant_components.values(), 
                                       labels=significant_components.keys(),
                                       autopct='%1.1f%%', startangle=90)
axes[0].set_title(f'ECM Composition: {target_application}', fontweight='bold')

# Plot 2: Comparison with natural tissues
tissue_moduli = [data['youngs_modulus_MPa'] for data in ecm_database.values()]
tissue_names_short = list(ecm_database.keys())

bars = axes[1].bar(tissue_names_short, np.log10(tissue_moduli), alpha=0.7, color='lightblue')
custom_bar = axes[1].bar([target_application], [np.log10(predicted_modulus)], 
                        color='red', alpha=0.8, label='Your Design')

axes[1].set_ylabel('Log₁₀(Young\'s Modulus [MPa])', fontweight='bold')
axes[1].set_title('Stiffness Comparison', fontweight='bold')
axes[1].tick_params(axis='x', rotation=45)
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\nTIP: Modify the composition values above and re-run to explore different designs!")

# Section 7: Analysis Questions for Students

Complete these questions based on your analysis results.

## **Question 1: ECM Component Analysis**
Based on the correlation analysis, which ECM components have the strongest positive and negative correlations with tissue stiffness? Explain why these relationships make biological sense.

*Write your answer here:*


---

## **Question 2: Tissue-Specific Compositions**
Compare the ECM compositions of bone, cartilage, and brain tissue. How do their compositions relate to their mechanical functions in the body?

*Write your answer here:*


---

## **Question 3: Custom ECM Design**
Using the ECM Designer tool, create compositions for these applications:

a) **Heart valve replacement** (needs flexibility and durability)
   - What composition did you choose and why?
   - What was the predicted Young's modulus?

*Write your answer here:*


b) **Bone graft material** (needs high stiffness)
   - What composition did you choose and why?
   - How does it compare to natural bone?

*Write your answer here:*


---

## **Question 4: Model Limitations**
The Young's modulus prediction model is simplified. What factors are not included that might affect real tissue mechanical properties?

*Write your answer here:*


---

## **Question 5: Clinical Applications**
From Chapter 2.2.4-2.2.5, we learned about natural vs synthetic ECM approaches. Based on your analysis, discuss the advantages and disadvantages of:

a) Using **decellularized ECM** from donor tissues
b) Creating **synthetic ECM** with defined compositions

*Write your analysis here:*


---

## **Question 6: Smart ECM Design**
From Section 2.2.6 on smart ECM mimics, how could you incorporate **stimuli-responsive** or **adaptive** properties into your ECM designs? Give a specific example.

*Write your ideas here:*



# Section 8: Extension Challenges (Optional)

For advanced students who want to explore further.

In [ ]:
# Extension Challenge 1: Multi-layer tissue design
# Design a gradient ECM composition (e.g., for skin with epidermis and dermis)

def create_gradient_ecm(layer1_composition, layer2_composition, n_layers=5):
    """
    Create a gradient between two ECM compositions.
    """
    gradient_layers = []
    
    for i in range(n_layers):
        alpha = i / (n_layers - 1)  # Interpolation factor
        layer_composition = {}
        
        for component in layer1_composition.keys():
            layer_composition[component] = (
                (1 - alpha) * layer1_composition[component] + 
                alpha * layer2_composition[component]
            )
        
        gradient_layers.append(layer_composition)
    
    return gradient_layers

# Example: Skin gradient from epidermis to dermis
epidermis_like = {
    'collagen_I': 0.60, 'collagen_II': 0.0, 'collagen_IV': 0.05,
    'elastin': 0.05, 'fibronectin': 0.10, 'laminin': 0.05,
    'hyaluronic_acid': 0.05, 'chondroitin_sulfate': 0.02,
    'dermatan_sulfate': 0.05, 'heparan_sulfate': 0.03,
    'hydroxyapatite': 0.0, 'water_content': 0.65
}

dermis_like = {
    'collagen_I': 0.80, 'collagen_II': 0.0, 'collagen_IV': 0.02,
    'elastin': 0.15, 'fibronectin': 0.02, 'laminin': 0.01,
    'hyaluronic_acid': 0.01, 'chondroitin_sulfate': 0.01,
    'dermatan_sulfate': 0.08, 'heparan_sulfate': 0.01,
    'hydroxyapatite': 0.0, 'water_content': 0.70
}

skin_gradient = create_gradient_ecm(epidermis_like, dermis_like, 5)

# Analyze gradient properties
print("Skin Gradient Analysis:")
print("Layer | Collagen I | Elastin | Water | Predicted Modulus (MPa)")
print("-" * 65)

for i, layer in enumerate(skin_gradient):
    modulus = predict_youngs_modulus(layer)
    print(f"{i+1:5} | {layer['collagen_I']:8.2f} | {layer['elastin']:6.2f} | "
          f"{layer['water_content']:5.2f} | {modulus:15.4f}")

# Visualize gradient
plt.figure(figsize=(12, 6))

# Plot composition gradients
layers = range(1, len(skin_gradient) + 1)
collagen_values = [layer['collagen_I'] for layer in skin_gradient]
elastin_values = [layer['elastin'] for layer in skin_gradient]
water_values = [layer['water_content'] for layer in skin_gradient]
modulus_values = [predict_youngs_modulus(layer) for layer in skin_gradient]

plt.subplot(1, 2, 1)
plt.plot(layers, collagen_values, 'b-o', label='Collagen I', linewidth=2)
plt.plot(layers, elastin_values, 'r-o', label='Elastin', linewidth=2)
plt.plot(layers, water_values, 'g-o', label='Water Content', linewidth=2)
plt.xlabel('Layer (Epidermis → Dermis)', fontweight='bold')
plt.ylabel('Relative Concentration', fontweight='bold')
plt.title('ECM Composition Gradient', fontweight='bold')
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
plt.plot(layers, np.log10(modulus_values), 'k-o', linewidth=3, markersize=8)
plt.xlabel('Layer (Epidermis → Dermis)', fontweight='bold')
plt.ylabel('Log₁₀(Young\'s Modulus [MPa])', fontweight='bold')
plt.title('Mechanical Property Gradient', fontweight='bold')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\nExtension Challenge Ideas:")
print("1. Create a bone-cartilage interface gradient")
print("2. Design ECM for blood vessel with tunica layers")
print("3. Model cornea ECM with transparency requirements")
print("4. Add time-dependent ECM remodeling simulation")

# Section 9: Summary and Key Takeaways

## **What You've Learned:**

1. **ECM Composition Analysis**: How different molecules contribute to tissue properties
2. **Structure-Function Relationships**: Why bone is stiff, cartilage is tough, and brain is soft
3. **Quantitative Modeling**: Using data to predict mechanical properties from composition
4. **ECM Design Principles**: How to engineer matrices for specific applications
5. **Advanced Python Skills**: Correlation analysis, data visualization, and predictive modeling

## **Connections to Chapter 2:**

- **Tables 2.3-2.5**: ECM component properties and tissue mechanical values
- **Section 2.2.1-2.2.3**: ECM composition, distribution, and tissue-specific organization
- **Section 2.2.4-2.2.6**: Natural vs synthetic ECM approaches and smart materials
- **Applications**: How ECM engineering supports scaffold-free biofabrication

## **Next Steps:**

Continue with **Exercise 3: Bioreactor Scale-up Analysis** to explore how to manufacture tissues at industrial scale.

---

## **Learning Assessment**

**Before finishing, make sure you can:**

- [ ] Identify which ECM components contribute to tissue stiffness vs flexibility
- [ ] Explain why bone, cartilage, and brain have such different mechanical properties
- [ ] Design ECM compositions for specific tissue engineering applications
- [ ] Interpret correlation data and understand structure-function relationships
- [ ] Connect ECM composition to clinical applications in regenerative medicine

**Excellent work completing Exercise 2!**